In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
from tslearn.clustering import KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import numpy as np
import glob
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import matplotlib.pyplot as plt
from tslearn.generators import random_walks
import tslearn.metrics as metrics
os.chdir(r'Z:\main\Data Mining\College Experience Study Dataset')
data = pd.read_csv('./EMA/general_ema.csv')
data_main = data[np.isnan(data['pam'])]
ind = list(data_main.index)
data_main = data.drop(index=ind,axis=0)
data_main = data_main.reset_index()
data_main = data_main.drop(['index'], axis=1)
EMA = data_main[['uid','day','pam', 'phq4_score', 'social_level', 'stress']]

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'phq4_score']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)

X = test
seed = 0
 

def test_elbow():
    global X,seed
    distortions = []
    X = TimeSeriesScalerMeanVariance(mu= 0.0 ,std= 1.0 ).fit_transform(X)
    #1 to 10 clusters 
    for i in  [20,30,40,50,60]:
        ks = KShape(n_clusters=i, n_init= 5 ,verbose= True ,random_state=seed)
        # Perform clustering calculation
        ks.fit(X)
        distortions.append(ks.inertia_)
    print(distortions)
    plt.plot([20,30,40,50,60], distortions, marker= 'o' )
    plt.xlabel( 'Number of clusters' )
    plt.ylabel( 'Distortion' )
    plt.show()
test_elbow()

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'phq4_score']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=40, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)

In [ ]:
plt.figure(dpi=100,figsize=(20,20))
for yi in range(40):
    plt.subplot(6, 7, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(40):
    plt.subplot(6, 7, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'phq4_score']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=20, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)

plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
label = {'uid' : patient_name,'EMA_cluster' : y_pred}
label = pd.DataFrame(data=label)
test = pd.merge(EMA, label, on=['uid'],how='outer')
test = test[['uid','day','phq4_score','EMA_cluster']]
recluster = {'EMA_cluster':[x for x in range(20)],'new_cluster':[0,0,2,2,4,1,5,3,6,1,7,3,8,1,3,2,2,9,3,2]}
recluster = pd.DataFrame(data=recluster)
test = pd.merge(test, recluster, on=['EMA_cluster'],how='outer')
test['EMA_cluster'] = test['new_cluster']
test = test[['uid','day','phq4_score','EMA_cluster']]
test.to_csv('final.EMA.csv',index=False)

基于之前做过的，情绪随时间变化的统计，找出一个周期的长度，以此为时间窗口进行采样和时间窗口聚类

In [ ]:
import sys
from importlib import reload
sys.path.insert(0,r'Z:\main\Data Mining\College Experience Study Dataset\EMA')

import emotion_visualization
reload(emotion_visualization)
name_list = list(EMA.columns)
name_list.remove('uid')
name_list.remove('day')
for x in name_list:
    emotion_visualization.emotion_visualization(x)

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'phq4_score']
test2 = pd.DataFrame(columns=[x for x in range(100)])
i = 0
while i < 1700:
    test3 = test.iloc[:,i:i+100]
    test3.columns = [x for x in range(100)]
    test2 = pd.concat([test2,test3],ignore_index=True)
    i = i + 75
test3 = test.iloc[:,-101:-1]
test3.columns = [x for x in range(100)]
test2 = pd.concat([test2,test3],ignore_index=True)
test2 = test2.fillna(0)
test2 = np.array(test2,dtype=np.float16)
test2 = np.nan_to_num(test2)
np.random.shuffle(test2)
test2 =  TimeSeriesScalerMeanVariance().fit_transform(test2)
sz = test2.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=3, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test2)

In [ ]:
plt.figure(dpi=100,figsize=(20,20))
for yi in range(3):
    plt.subplot(3, 4, 1 + yi)
    for xx in test2[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(3):
    plt.subplot(3, 4, 1 + yi)
    for xx in test2[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
test2 = pd.DataFrame(columns=[x for x in range(500)])
i = 0
while i < 1200:
    test3 = test.iloc[:,i:i+500]
    test3.columns = [x for x in range(500)]
    test2 = pd.concat([test2,test3],ignore_index=True)
    i = i + 400
test3 = test.iloc[:,-501:-1]
test3.columns = [x for x in range(500)]
test2 = pd.concat([test2,test3],ignore_index=True)
test2 = test2.fillna(0)
test2 = np.array(test2,dtype=np.float16)
test2 = np.nan_to_num(test2)
np.random.shuffle(test2)
test2 =  TimeSeriesScalerMeanVariance().fit_transform(test2)
sz = test2.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=4, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test2)

In [ ]:
plt.figure(dpi=100,figsize=(20,20))
for yi in range(4):
    plt.subplot(2, 2, 1 + yi)
    for xx in test2[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(4):
    plt.subplot(2, 2, 1 + yi)
    for xx in test2[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
EMA

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'pam']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=20, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
label = {'uid' : patient_name,'EMA_cluster' : y_pred}
label = pd.DataFrame(data=label)
test = pd.merge(EMA, label, on=['uid'],how='outer')
test = test[['uid','day','pam','EMA_cluster']]
recluster = {'EMA_cluster':[x for x in range(20)],'new_cluster':[4,4,1,1,2,3,3,0,0,3,3,5,1,3,6,0,0,7,1,1]}
recluster = pd.DataFrame(data=recluster)
test = pd.merge(test, recluster, on=['EMA_cluster'],how='outer')
test['EMA_cluster'] = test['new_cluster']
test = test[['uid','day','pam','EMA_cluster']]
test.to_csv('final.EMA.pam.csv',index=False)

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'social_level']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=20, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
label = {'uid' : patient_name,'EMA_cluster' : y_pred}
label = pd.DataFrame(data=label)
test = pd.merge(EMA, label, on=['uid'],how='outer')
test = test[['uid','day','social_level','EMA_cluster']]
recluster = {'EMA_cluster':[x for x in range(20)],'new_cluster':[3,3,0,1,1,2,2,1,1,2,3,2,1,2,4,1,0,4,1,0]}
recluster = pd.DataFrame(data=recluster)
test = pd.merge(test, recluster, on=['EMA_cluster'],how='outer')
test['EMA_cluster'] = test['new_cluster']
test = test[['uid','day','social_level','EMA_cluster']]
test.to_csv('final.EMA.social.csv',index=False)

In [ ]:
patient_name = list(set(list(EMA['uid'])))
time_point = sorted(list(set(EMA['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(EMA)):
    test.loc[EMA.loc[x,'uid'],EMA.loc[x,'day']] = EMA.loc[x,'stress']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=20, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()
plt.figure(dpi=100,figsize=(20,20))
for yi in range(20):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-1, 13)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
label = {'uid' : patient_name,'EMA_cluster' : y_pred}
label = pd.DataFrame(data=label)
test = pd.merge(EMA, label, on=['uid'],how='outer')
test = test[['uid','day','stress','EMA_cluster']]
recluster = {'EMA_cluster':[x for x in range(20)],'new_cluster':[3,3,1,1,0,2,2,0,0,2,3,4,1,2,5,0,0,6,1,1]}
recluster = pd.DataFrame(data=recluster)
test = pd.merge(test, recluster, on=['EMA_cluster'],how='outer')
test['EMA_cluster'] = test['new_cluster']
test = test[['uid','day','stress','EMA_cluster']]
test.to_csv('final.EMA.stress.csv',index=False)